<a href="https://colab.research.google.com/github/maktaurus/ML-Work/blob/main/Torch_Notebooks/Text_Classification_Pre_Trained_Torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# For Imbalanced datasets, calculate weights as per below formula and pass in loss function:
# **For Binary Classification:**
weight_for_class_i = total_samples / (num_samples_in_class_i * num_classes)

weight_for_class_0 = 1000 / (900 * 2) = 0.5556

weight_for_class_1 = 1000 / (100 * 2) = 5.0000

# **Multi-class Classification**
To address class imbalance, we can assign a weight to each class that inversely proportional to its frequency. The weight of each class can be calculated as follows:

Class A: 1000 / 100 = 10

Class B: 1000 / 800 = 1.25

Class C: 1000 / 100 = 10

import torch

weights = torch.tensor([10, 1.25, 10])

loss_fn = torch.nn.CrossEntropyLoss(weight=weights)

In [1]:
pip install -q pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.2/866.2 kB 1.3 MB/s eta 0:00:00


In [2]:
import torch
import pytorch_lightning as pl
import pandas as pd
import numpy as np
import zipfile

In [3]:
!kaggle datasets download saurabhshahane/ecommerce-text-classification

Dataset URL: https://www.kaggle.com/datasets/saurabhshahane/ecommerce-text-classification
License(s): Attribution 4.0 International (CC BY 4.0)
 64% 5.00M/7.86M [00:00<00:00, 34.3MB/s]
100% 7.86M/7.86M [00:00<00:00, 51.0MB/s]


In [4]:
zipfile.ZipFile("/content/ecommerce-text-classification.zip").extractall()

In [6]:
data = pd.read_csv("/content/ecommerceDataset.csv",names=["cat","desc"])
data.head()

,cat,desc
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


In [7]:
data.dropna(inplace=True)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50424 entries, 0 to 50424
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cat     50424 non-null  object
 1   desc    50424 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB


In [9]:
labels = sorted(data["cat"].unique())
labels = {x:y for y,x in enumerate(labels)}
labels

{'Books': 0, 'Clothing & Accessories': 1, 'Electronics': 2, 'Household': 3}

In [10]:
data["cat"] = data["cat"].map(labels)

In [11]:
data.head()

,cat,desc
0,3,Paper Plane Design Framed Wall Hanging Motivat...
1,3,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,3,SAF 'UV Textured Modern Art Print Framed' Pain...
3,3,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,3,Incredible Gifts India Wooden Happy Birthday U...


In [12]:

data = data.sample(frac=1).reset_index(drop=True)

In [34]:
test = data.sample(100)
data = data.drop(test.index).reset_index(drop=True)
test.reset_index(drop=True,inplace=True)

In [15]:
data = data[["desc","cat"]]

In [16]:
data.head()

,desc,cat
0,Quality Assure Roland Beige & White 9 Seater F...,3
1,Westinghouse WFL545 Select Series Seasoned Cas...,3
2,DECORE Baby Boy's Kurta Pyjama Display a grac ...,1
3,Heinemann Explore Science Student Book (Cambri...,0
4,Larah By Borosil Helena Glass Cup and Saucer S...,3


In [17]:
class MyDataset(torch.utils.data.Dataset):
  def __init__(self,data):
    super().__init__()
    self.data = data

  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    text = self.data.iloc[idx,0]
    label = self.data.iloc[idx,1]
    return text,label
df = MyDataset(data)

In [18]:
for x in df:
  print(x)
  break

('Quality Assure Roland Beige & White 9 Seater Fabric Sofa Set with Four Puffy A sectional sofa is modular. It usually includes at least two pieces that can be arranged in multiple configurations. A sectional sofa often has the option to choose the arm position as either left or right, with two or no arms. The key benefit of a sectional sofa is the ability to tailor its components to a range of spaces. Although sectional sofas were manufactured as early as the Victorian era, they did not become prominent until the mid-1950s, and they continue to be refined to the present day. At its most extreme, the Lava Sofa by Quality Assure Furniture -- a soft, flowing sectional -- can be arranged in any number of ways including crossed, paralleled or stacked. For More Details Kindly Contact Us on :- Qualityassurefurniture@gmail.com.', 3)


In [19]:
train,val = torch.utils.data.random_split(df,[0.9,0.1])

In [20]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [21]:
for x in df:
  print(x[0])
  tokens = tokenizer(x[0],max_length=256,padding="max_length",truncation=True,return_tensors="pt")
  print(tokens["input_ids"])
  print(tokens["attention_mask"].shape)
  break

Quality Assure Roland Beige & White 9 Seater Fabric Sofa Set with Four Puffy A sectional sofa is modular. It usually includes at least two pieces that can be arranged in multiple configurations. A sectional sofa often has the option to choose the arm position as either left or right, with two or no arms. The key benefit of a sectional sofa is the ability to tailor its components to a range of spaces. Although sectional sofas were manufactured as early as the Victorian era, they did not become prominent until the mid-1950s, and they continue to be refined to the present day. At its most extreme, the Lava Sofa by Quality Assure Furniture -- a soft, flowing sectional -- can be arranged in any number of ways including crossed, paralleled or stacked. For More Details Kindly Contact Us on :- Qualityassurefurniture@gmail.com.
tensor([[  101,  3737, 14306,  8262, 28799,  1004,  2317,  1023, 23392,  8313,
         10682,  2275,  2007,  2176, 23893,  2100,  1037, 27197, 10682,  2003,
         19

In [22]:
def collate_fn(batch):
  tokens = [tokenizer(str(x[0]),max_length=256,padding="max_length",truncation=True,return_tensors="pt") for x in batch]
  input_ids = torch.stack([x["input_ids"].squeeze(0) for x in tokens])
  attention_mask = torch.stack([x["attention_mask"].squeeze(0) for x in tokens])
  labels = torch.tensor([x[1] for x in batch])
  return input_ids,attention_mask,labels

In [24]:
train_df = torch.utils.data.DataLoader(train,batch_size=32,shuffle=True,collate_fn=collate_fn)
val_df = torch.utils.data.DataLoader(val,batch_size=32,collate_fn=collate_fn)

In [25]:
for x in train_df:
  print(x[0])
  print(x[1].shape)
  print(x[2].shape)
  break

tensor([[  101,  5061, 12269,  ...,     0,     0,     0],
        [  101, 14021, 12044,  ...,     0,     0,     0],
        [  101,  2188,  2135,  ...,     0,     0,     0],
        ...,
        [  101, 16432,  1997,  ...,     0,     0,     0],
        [  101,  1058,  1011,  ...,  3430,  2008,   102],
        [  101,  8814, 12399,  ...,     0,     0,     0]])
torch.Size([32, 256])
torch.Size([32])


In [26]:
model = BertModel.from_pretrained("bert-base-uncased")
model

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [27]:
for param in model.parameters():
  param.requires_grad = False


In [31]:
class RobModel(torch.nn.Module):
  def __init__(self,model):
    super().__init__()
    self.model = model
    self.linear1 = torch.nn.Linear(768,768)
    self.linear2 = torch.nn.Linear(768,768)
    self.linear3 = torch.nn.Linear(768,4)
    # self.relu = torch.nn.ReLU()

  def forward(self,input_ids,attention_mask):
    x = self.model(input_ids=input_ids,attention_mask=attention_mask)
    x = self.linear1(x[1])
    x = self.linear2(x)
    # x = self.relu(x)
    x = self.linear3(x)
    return x

robmodel = RobModel(model)
robmodel

RobModel(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [ ]:
for x in train_df:
  print(robmodel(x[0],x[1]).shape)
  print(x[2].shape)
  break

torch.Size([12, 27])
torch.Size([12])


In [ ]:
# Example of target with class indices
# loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(input)
print(target)
# output = loss(input, target)
# output.backward()
# # Example of target with class probabilities
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.randn(3, 5).softmax(dim=1)
# output = loss(input, target)
# output.backward()

tensor([[ 1.2102,  0.3986,  0.9535,  1.3105,  0.4278],
        [-0.0909,  0.6695, -0.5412, -0.4237, -0.3870],
        [ 0.3253, -0.5563, -1.3185,  1.7519, -0.2965]], requires_grad=True)
tensor([3, 0, 2])


In [32]:
class LightningModel(pl.LightningModule):
  def __init__(self,model):
    super().__init__()
    self.model = model
    self.loss = torch.nn.CrossEntropyLoss()

  def forward(self,input_ids,attention_mask):
    return self.model(input_ids,attention_mask)

  def training_step(self,batch,batch_idx):
    input_ids,attention_mask,labels = batch
    out = self.model(input_ids,attention_mask)
    loss = self.loss(out,labels.long())
    self.log("train_loss",loss,on_step=True,on_epoch=True,prog_bar=True,logger=True)
    return loss

  def validation_step(self,batch,batch_idx):
    input_ids,attention_mask,labels = batch
    out = self.model(input_ids,attention_mask)
    loss = self.loss(out,labels.long())
    self.log("val_loss",loss,on_step=True,on_epoch=True,prog_bar=True,logger=True)
    return loss

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(),lr=0.001)

In [33]:
pl_model = LightningModel(robmodel).train()
trainer = pl.Trainer(max_epochs=2)
trainer.fit(pl_model,train_df,val_df)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params | Mode 
---------------------------------------------------
0 | model | RobModel         | 110 M  | train
1 | loss  | CrossEntropyLoss | 0      | train
---------------------------------------------------
1.2 M     Trainable params
109 M     Non-trainable params
110 M     Total params
442.666   Total estimated model params size (MB)
233       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=2` reached.


In [35]:
test.head()

,desc,cat
0,SanDisk Ultra Dual 64GB USB 3.0 OTG Pen Drive ...,2
1,Non Resident Idiot,0
2,Evynza Micro USB Tough & Unbreakable Braided C...,2
3,EthnicJunction Rajasthani Mirror Work Chanderi...,1
4,Butane Gas Blow Torch Auto Ignition Outdoor We...,3


In [36]:
test_df = MyDataset(test)

In [37]:
test_df.data

,desc,cat
0,SanDisk Ultra Dual 64GB USB 3.0 OTG Pen Drive ...,2
1,Non Resident Idiot,0
2,Evynza Micro USB Tough & Unbreakable Braided C...,2
3,EthnicJunction Rajasthani Mirror Work Chanderi...,1
4,Butane Gas Blow Torch Auto Ignition Outdoor We...,3
...,...,...
95,Alfa Children Knob (Micky) Cartoon Shape Drawe...,3
96,Milton Thermosteel Classic Carafe Tea / Coffee...,3
97,Magicwand 4 Big Metal Beyblades With Led Light...,0
98,Bank PO Quantitative Aptitude Solved Papers Ab...,0


In [38]:
pl_model.eval()

LightningModel(
  (model): RobModel(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
    

In [41]:
for i,x in enumerate(test_df):
  if i == 30:
    break
  tokens = tokenizer(x[0],max_length=256,padding="max_length",truncation=True,return_tensors="pt")
  # with torch.no_grad():
  out = pl_model(tokens["input_ids"],tokens["attention_mask"])
  soft = torch.nn.Softmax(dim=-1)
  out = soft(out)
  out = torch.argmax(out,dim=-1)
  print(out)
  print(x[1])
  # print(labels[x[1]])
  # print(x[0])


tensor([2])
2
tensor([0])
0
tensor([2])
2
tensor([1])
1
tensor([3])
3
tensor([1])
3
tensor([2])
2
tensor([1])
0
tensor([0])
0
tensor([0])
0
tensor([0])
0
tensor([1])
1
tensor([1])
1
tensor([3])
3
tensor([1])
1
tensor([2])
2
tensor([2])
2
tensor([1])
1
tensor([0])
0
tensor([1])
1
tensor([3])
3
tensor([3])
3
tensor([3])
3
tensor([1])
1
tensor([1])
3
tensor([3])
3
tensor([3])
3
tensor([0])
0
tensor([0])
0
tensor([1])
0


In [40]:
labels

{'Books': 0, 'Clothing & Accessories': 1, 'Electronics': 2, 'Household': 3}

In [ ]:
torch.cuda.empty_cache()